In [2]:
from PySide6.QtSql import QSqlDatabase
from mysql import connector

conn = connector.connect(    
    host='localhost',
    user='root',
    password='root')

cursor = conn.cursor()


In [4]:
# 创建数据库

conn.start_transaction()

try:
    sql = 'CREATE SCHEMA IF NOT EXISTS attendance_system'
    cursor.execute(sql)
    conn.commit()
except Exception as e:
    conn.rollback()
    print("Error creating database:", e)


In [5]:
# 该格子用于创建对应的表。包括学生表，教师表，课程表，课程学生表，考勤表

# 开始事务
conn.start_transaction()

try:

    # Switch to the new schema
    cursor.execute("USE attendance_system")

    # Create the teacher table
    create_teacher_table = """
    CREATE TABLE IF NOT EXISTS teacher (
        name VARCHAR(100) NOT NULL,
        username VARCHAR(100) NOT NULL PRIMARY KEY,
        password VARCHAR(100) NOT NULL
    )
    """
    cursor.execute(create_teacher_table)

    # Create the student table
    create_student_table = """
    CREATE TABLE IF NOT EXISTS student (
        id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        gender ENUM('男', '女') NULL,
        age CHAR(3) NULL,
        name VARCHAR(100) NULL,
        is_face_collected BOOLEAN NOT NULL DEFAULT FALSE
    )
    """
    cursor.execute(create_student_table)

    # Create the course table
    create_course_table = """
    CREATE TABLE IF NOT EXISTS course (
        course_name VARCHAR(100) NOT NULL PRIMARY KEY,
        teacher_username VARCHAR(100) NOT NULL,
        FOREIGN KEY (teacher_username) REFERENCES teacher(username) On DELETE CASCADE
    )
    """
    cursor.execute(create_course_table)

    # Create the course_student table
    create_course_student_table = """
    CREATE TABLE IF NOT EXISTS course_student (
        course_name VARCHAR(100) NOT NULL,
        student_id INT NOT NULL,
        PRIMARY KEY (course_name, student_id),
        FOREIGN KEY (course_name) REFERENCES course(course_name) ON DELETE CASCADE,
        FOREIGN KEY (student_id) REFERENCES student(id) ON DELETE CASCADE
    )
    """
    cursor.execute(create_course_student_table)

    # Create the attendance_info table
    create_attendance_info_table = """
    CREATE TABLE IF NOT EXISTS attendance_info (
        date DATE NOT NULL,
        stu_id INT NOT NULL,
        course_name VARCHAR(100) NOT NULL,
        is_attendance BOOLEAN NOT NULL,
        PRIMARY KEY (date, stu_id, course_name),
        FOREIGN KEY (stu_id) REFERENCES student(id) ON DELETE CASCADE,
        FOREIGN KEY (course_name) REFERENCES course(course_name) ON DELETE CASCADE
    )
    """
    cursor.execute(create_attendance_info_table)

    # Commit the changes
    conn.commit()
except Exception as e:
    # 如果发生错误，回滚事务
    conn.rollback()
    print(f"An error occurred: {e}")

In [7]:
# 生成数据
import random
from faker import Faker
from datetime import datetime, timedelta


cursor.execute("USE attendance_system")
fake = Faker("zh_CN")


conn.start_transaction()

try:
    # 插入老师数据
    teachers = []  # Use a list to store unique teachers
    existing_usernames = set()  # Track unique usernames
    while len(teachers) < 20:
        name = fake.name()
        username = name
        password = name
        if username not in existing_usernames:  # Ensure uniqueness
            teachers.append((name, username, password))
            existing_usernames.add(username)

    cursor.executemany(
        "INSERT INTO teacher (name, username, password) VALUES (%s, %s, %s)", teachers
    )

    # 插入课程数据
    # 定义50个真实的中文课程名称
    course_names = [
        # 理工类课程
        "高等数学", "线性代数", "概率论与数理统计", "大学物理", "工程力学",
        "电路分析基础", "模拟电子技术", "数字电子技术", "信号与系统", "自动控制原理",
        "数据结构与算法", "计算机组成原理", "操作系统", "计算机网络", "数据库系统原理",
        "软件工程", "人工智能导论", "机器学习", "深度学习", "计算机视觉",
        
        # 文科类课程
        "中国近现代史纲要", "马克思主义基本原理", "思想道德修养与法律基础", "毛泽东思想与中国特色社会主义理论体系概论", "形势与政策",
        "大学英语", "学术英语写作", "中国文学史", "世界文学史", "新闻采访与写作",
        
        # 商科类课程
        "微观经济学", "宏观经济学", "管理学原理", "会计学原理", "财务管理",
        "市场营销学", "国际贸易实务", "电子商务概论", "企业战略管理", "人力资源管理",
        
        # 医学/生物类课程
        "人体解剖学", "生理学", "病理学", "药理学", "免疫学基础",
        "生物化学", "分子生物学", "遗传学", "微生物学", "有机化学"
    ]

    # 替换原有的课程生成代码
    courses = []
    for i in range(50):
        course_name = course_names[i]  # 使用预定义的课程名称
        teacher_username = random.choice(teachers)[1]
        courses.append((course_name, teacher_username))

    cursor.executemany(
        "INSERT INTO course (course_name, teacher_username) VALUES (%s, %s)", courses
    )

    # 插入学生数据
    students = []
    for i in range(500):
        name = fake.name()
        gender = random.choice(["男", "女"])
        age = str(random.randint(18, 25))
        is_face_collected = False
        students.append((gender, age, name, is_face_collected))

    cursor.executemany(
        "INSERT INTO student (gender, age, name, is_face_collected) VALUES (%s, %s, %s, %s)",
        students,
    )

    # 获取学生的实际 ID
    cursor.execute("SELECT id FROM student")
    student_ids = [row[0] for row in cursor.fetchall()]

    # 插入课程学生数据
    course_student = []
    for student_id in student_ids:
        enrolled_courses = random.sample(courses, random.randint(5, 8))
        for course in enrolled_courses:
            course_student.append((course[0], student_id))

    cursor.executemany(
        "INSERT INTO course_student (course_name, student_id) VALUES (%s, %s)", course_student
    )

    # 插入签到表数据
    attendance_info = []
    start_date = datetime(2025, 3, 20)
    end_date = datetime(2025, 3, 26)
    date_range = (end_date - start_date).days + 1

    for day in range(date_range):
        current_date = start_date + timedelta(days=day)
        for course in courses:
            course_name = course[0]
            enrolled_students = [
                student_id
                for course_name_, student_id in course_student
                if course_name_ == course_name
            ]
            attendance_count = int(len(enrolled_students) * random.uniform(0.9, 0.99))
            attending_students = random.sample(enrolled_students, attendance_count)
            for student_id in attending_students:
                attendance_info.append((current_date, student_id, course_name, True))

    cursor.executemany(
        "INSERT INTO attendance_info (date, stu_id, course_name, is_attendance) VALUES (%s, %s, %s, %s)",
        attendance_info,
    )

    # 最后提交事务
    conn.commit()
    print("成功生成数据库数据")
except Exception as e:
    conn.rollback()
    print(f"An error occurred: {e}")


成功生成数据库数据


In [6]:
cursor.execute("""
SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = 'attendance_system'
""")
tables = cursor.fetchall()
for table in tables:
    table_name = table[0]
    cursor.execute(f"SHOW CREATE TABLE {table_name}")
    create_table_query = cursor.fetchone()
    print(f"Table: {table_name}")
    print(create_table_query[1])
    print("\n")

Table: attendance_info
CREATE TABLE `attendance_info` (
  `date` date NOT NULL,
  `stu_id` int NOT NULL,
  `course_name` varchar(100) NOT NULL,
  `is_attendance` tinyint(1) NOT NULL,
  PRIMARY KEY (`date`,`stu_id`,`course_name`),
  KEY `stu_id` (`stu_id`),
  KEY `course_name` (`course_name`),
  CONSTRAINT `attendance_info_ibfk_1` FOREIGN KEY (`stu_id`) REFERENCES `student` (`id`) ON DELETE CASCADE,
  CONSTRAINT `attendance_info_ibfk_2` FOREIGN KEY (`course_name`) REFERENCES `course` (`course_name`) ON DELETE CASCADE
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci


Table: course
CREATE TABLE `course` (
  `course_name` varchar(100) NOT NULL,
  `teacher_username` varchar(100) NOT NULL,
  PRIMARY KEY (`course_name`),
  KEY `teacher_username` (`teacher_username`),
  CONSTRAINT `course_ibfk_1` FOREIGN KEY (`teacher_username`) REFERENCES `teacher` (`username`) ON DELETE CASCADE
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci


Table: course_student
CREAT

数据生成完成！


In [12]:
# 删除表里的所欲数据

cursor.execute("""
SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = 'attendance_system'
""")
tables = cursor.fetchall()
# Delete the content of each table
try:
    try:
        conn.rollback()
        print("回滚了之前可能存在的事务")
    except:
        pass
    # Start transaction
    conn.start_transaction()

    # Disable foreign key checks
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0")

    for table in tables:
        table_name = table[0]
        cursor.execute(f"DELETE FROM {table_name}")
        print(f"All content deleted from table: {table_name}")

    # Enable foreign key checks
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1")

    # Commit transaction
    conn.commit()
except Exception as e:
    # Rollback transaction in case of error
    conn.rollback()
    print(f"An error occurred: {e}")


回滚了之前可能存在的事务
All content deleted from table: attendance_info
All content deleted from table: course
All content deleted from table: course_student
All content deleted from table: student
All content deleted from table: teacher
